In [1]:
%env TORCH_SHOW_DISPATCH_TRACE=1
import torch
import torch_xla
import torch_xla.runtime
device = torch_xla.device()

env: TORCH_SHOW_DISPATCH_TRACE=1


 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar], key=[CPU]
 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar], key=[CPU]
 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar], key=[CPU]
 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar]

In [9]:
import torch
import torch_xla
import torch_xla.runtime
device = torch_xla.device()
x = torch.randn(3, 3, requires_grad=False, device=device)
y = torch.randn(3, 3, requires_grad=False, device=device)
torch_xla.sync()
out = torch.einsum('...n,mn->...m', x, y)
print(torch_xla._XLAC._get_xla_tensors_text([out]))

IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3,1]{2,1,0} aten::view(%0), xla_shape=f32[3,3,1]{2,1,0}
  %2 = f32[1,3,3]{0,2,1} aten::permute(%1), xla_shape=f32[1,3,3]{0,2,1}
  %3 = f32[3,3,1]{2,0,1} aten::permute(%2), xla_shape=f32[3,3,1]{2,0,1}
  %4 = f32[1,3,3]{2,1,0} aten::view(%3), xla_shape=f32[1,3,3]{2,1,0}
  %5 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %6 = f32[3,3,1]{2,1,0} aten::view(%5), xla_shape=f32[3,3,1]{2,1,0}
  %7 = f32[3,1,3]{1,2,0} aten::permute(%6), xla_shape=f32[3,1,3]{1,2,0}
  %8 = f32[3,3,1]{2,1,0} aten::permute(%7), xla_shape=f32[3,3,1]{2,1,0}
  %9 = f32[1,3,3]{2,1,0} aten::view(%8), xla_shape=f32[1,3,3]{2,1,0}
  %10 = f32[1,3,3]{2,1,0} aten::matmul(%9, %4), xla_shape=f32[1,3,3]{2,1,0}
  %11 = f32[3,1,3]{2,1,0} aten::view(%10), xla_shape=f32[3,1,3]{2,1,0}
  %12 = f32[3,3,1]{1,2,0} aten::permute(%11), xla_shape=f32[3,3,1]{1,2,0}
  %13 = f32[3,3]{1,0} aten::view(%12), xla_shape=f32[3,3]{1,0}, ROOT=0
}



In [10]:
import torch
import torch_xla
import torch_xla.runtime
device = torch_xla.device()
x = torch.randn(3, 3, requires_grad=True, device=device)
y = torch.randn(3, 3, requires_grad=True, device=device)
torch_xla.sync()
out = torch.einsum('...n,mn->...m', x, y)
print(torch_xla._XLAC._get_xla_tensors_text([out]))

IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3,1]{2,1,0} aten::view(%0), xla_shape=f32[3,3,1]{2,1,0}
  %2 = f32[1,3,3]{0,2,1} aten::permute(%1), xla_shape=f32[1,3,3]{0,2,1}
  %3 = f32[3,3,1]{2,0,1} aten::permute(%2), xla_shape=f32[3,3,1]{2,0,1}
  %4 = f32[1,3,3]{2,1,0} aten::view(%3), xla_shape=f32[1,3,3]{2,1,0}
  %5 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %6 = f32[3,3,1]{2,1,0} aten::view(%5), xla_shape=f32[3,3,1]{2,1,0}
  %7 = f32[3,1,3]{1,2,0} aten::permute(%6), xla_shape=f32[3,1,3]{1,2,0}
  %8 = f32[3,3,1]{2,1,0} aten::permute(%7), xla_shape=f32[3,3,1]{2,1,0}
  %9 = f32[1,3,3]{2,1,0} aten::view(%8), xla_shape=f32[1,3,3]{2,1,0}
  %10 = f32[1,3,3]{2,1,0} aten::matmul(%9, %4), xla_shape=f32[1,3,3]{2,1,0}
  %11 = f32[3,1,3]{2,1,0} aten::view(%10), xla_shape=f32[3,1,3]{2,1,0}
  %12 = f32[3,3,1]{1,2,0} aten::permute(%11), xla_shape=f32[3,3,1]{1,2,0}
  %13 = f32[3,3]{1,0} aten::view(%12), xla_shape=f32[3,3]{1,0}, ROOT=0
}



In [11]:
print("-- Does creating with gradient enabled make a difference?")

print("-- Test1")
x_1= torch.randn(3, 3, requires_grad=False).to('xla')
y_1= torch.randn(3, 3, requires_grad=False).to('xla')
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test2")
x_2= torch.randn(3, 3, requires_grad=True)
y_2= torch.randn(3, 3, requires_grad=True)
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test3")
x_1= torch.randn(3, 3, requires_grad=False, device=device)
y_1= torch.randn(3, 3, requires_grad=False, device=device)
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test4")
x_2= torch.randn(3, 3, requires_grad=False, device=device)
y_2= torch.randn(3, 3, requires_grad=False, device=device)
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

-- Does creating with gradient enabled make a difference?
-- Test1
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3,1]{2,1,0} aten::view(%0), xla_shape=f32[3,3,1]{2,1,0}
  %2 = f32[1,3,3]{0,1,2} aten::permute(%1), xla_shape=f32[1,3,3]{0,1,2}
  %3 = f32[3,3,1]{2,1,0} aten::permute(%2), xla_shape=f32[3,3,1]{2,1,0}
  %4 = f32[1,3,3]{2,1,0} aten::view(%3), xla_shape=f32[1,3,3]{2,1,0}
  %5 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %6 = f32[3,3,1]{2,1,0} aten::view(%5), xla_shape=f32[3,3,1]{2,1,0}
  %7 = f32[3,1,3]{1,2,0} aten::permute(%6), xla_shape=f32[3,1,3]{1,2,0}
  %8 = f32[3,3,1]{2,1,0} aten::permute(%7), xla_shape=f32[3,3,1]{2,1,0}
  %9 = f32[1,3,3]{2,1,0} aten::view(%8), xla_shape=f32[1,3,3]{2,1,0}
  %10 = f32[1,3,3]{2,1,0} aten::matmul(%9, %4), xla_shape=f32[1,3,3]{2,1,0}
  %11 = f32[3,1,3]{2,1,0} aten::view(%10), xla_shape=f32[3,1,3]{2,1,0}
  %12 = f32[3,3,1]{1,2,0} aten::permute(%11), xla_shape=f32[3,3,1]{1,2,0}
  %13 = f32[3

It doesn't seem like gradient made a difference. However, device creation mattered

In [12]:
print("-- Is there a different if we create in the CPU, and then transfer to XLA?")
print("-- Check 1")
x_1= torch.randn(3, 3, requires_grad=False).to('xla')
y_1= torch.randn(3, 3, requires_grad=False).to('xla')
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 2")
x_1= torch.randn(3, 3, requires_grad=True).to('xla')
y_1= torch.randn(3, 3, requires_grad=True).to('xla')
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 3")
with torch_xla.runtime.xla_device():
  x_1= torch.randn(3, 3, requires_grad=False).to('xla')
  y_1= torch.randn(3, 3, requires_grad=False).to('xla')
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 4")
with torch_xla.runtime.xla_device():
  x_1= torch.randn(3, 3, requires_grad=True).to('xla')
  y_1= torch.randn(3, 3, requires_grad=True).to('xla')
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 5")
x_1= torch.randn(3, 3, requires_grad=True)
y_1= torch.randn(3, 3, requires_grad=True)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1.to('xla'), y_1.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))



-- Is there a different if we create in the CPU, and then transfer to XLA?
-- Check 1
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3,1]{2,1,0} aten::view(%0), xla_shape=f32[3,3,1]{2,1,0}
  %2 = f32[1,3,3]{0,1,2} aten::permute(%1), xla_shape=f32[1,3,3]{0,1,2}
  %3 = f32[3,3,1]{2,1,0} aten::permute(%2), xla_shape=f32[3,3,1]{2,1,0}
  %4 = f32[1,3,3]{2,1,0} aten::view(%3), xla_shape=f32[1,3,3]{2,1,0}
  %5 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %6 = f32[3,3,1]{2,1,0} aten::view(%5), xla_shape=f32[3,3,1]{2,1,0}
  %7 = f32[3,1,3]{1,2,0} aten::permute(%6), xla_shape=f32[3,1,3]{1,2,0}
  %8 = f32[3,3,1]{2,1,0} aten::permute(%7), xla_shape=f32[3,3,1]{2,1,0}
  %9 = f32[1,3,3]{2,1,0} aten::view(%8), xla_shape=f32[1,3,3]{2,1,0}
  %10 = f32[1,3,3]{2,1,0} aten::matmul(%9, %4), xla_shape=f32[1,3,3]{2,1,0}
  %11 = f32[3,1,3]{2,1,0} aten::view(%10), xla_shape=f32[3,1,3]{2,1,0}
  %12 = f32[3,3,1]{1,2,0} aten::permute(%11), xla_shape=f32[3,3,1]{1

Conclusion:
It seems like creating the tensor in the xla_device is causing the failure. If we create it, and then transition to the TPU, it succeeds

In [13]:
print("-- Does creating with gradient enabled make a difference?")

print("-- Test1")
x_1= torch.randn(3, 3, requires_grad=False).to('xla')
y_1= torch.randn(3, 3, requires_grad=False).to('xla')
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test2")
x_2= torch.randn(3, 3, requires_grad=True)
y_2= torch.randn(3, 3, requires_grad=True)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test3")
x_1= torch.randn(3, 3, requires_grad=False, device=device)
y_1= torch.randn(3, 3, requires_grad=False, device=device)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test4")
x_2= torch.randn(3, 3, requires_grad=False, device=device)
y_2= torch.randn(3, 3, requires_grad=False, device=device)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

-- Does creating with gradient enabled make a difference?
-- Test1
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3,1]{2,1,0} aten::view(%0), xla_shape=f32[3,3,1]{2,1,0}
  %2 = f32[1,3,3]{0,1,2} aten::permute(%1), xla_shape=f32[1,3,3]{0,1,2}
  %3 = f32[3,3,1]{2,1,0} aten::permute(%2), xla_shape=f32[3,3,1]{2,1,0}
  %4 = f32[1,3,3]{2,1,0} aten::view(%3), xla_shape=f32[1,3,3]{2,1,0}
  %5 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %6 = f32[3,3,1]{2,1,0} aten::view(%5), xla_shape=f32[3,3,1]{2,1,0}
  %7 = f32[3,1,3]{1,2,0} aten::permute(%6), xla_shape=f32[3,1,3]{1,2,0}
  %8 = f32[3,3,1]{2,1,0} aten::permute(%7), xla_shape=f32[3,3,1]{2,1,0}
  %9 = f32[1,3,3]{2,1,0} aten::view(%8), xla_shape=f32[1,3,3]{2,1,0}
  %10 = f32[1,3,3]{2,1,0} aten::matmul(%9, %4), xla_shape=f32[1,3,3]{2,1,0}
  %11 = f32[3,1,3]{2,1,0} aten::view(%10), xla_shape=f32[3,1,3]{2,1,0}
  %12 = f32[3,3,1]{1,2,0} aten::permute(%11), xla_shape=f32[3,3,1]{1,2,0}
  %13 = f32[3

In [2]:
%env TORCH_SHOW_DISPATCH_TRACE=1
import torch
from torch import Tensor
from torch.library import custom_op
import time
import torch_xla
import torch_xla.runtime
device = torch_xla.device()

@custom_op("xla::custom_linear_forward123", schema="(Tensor input, Tensor weight) -> Tensor", mutates_args=())
def custom_linear_forward123(input: Tensor, weight: Tensor):
    return torch.einsum('...n,mn->...m', input, weight)

X = torch.zeros(3, 3, requires_grad=False, device='xla')
Y = torch.zeros(3, 3, requires_grad=False, device='xla')

def test_lowering(func):
  time.sleep(1)
  out = func(X, Y)
  time.sleep(1)
  ir = torch_xla._XLAC._get_xla_tensors_text([out])
  if 'einsum' not in ir:
    print("!!!!!!!!!!WRONG!!!!!!!!!!! Did not find einsum in lowering")
    print("IR:")
    print(ir)
  else:
    print("OK")

env: TORCH_SHOW_DISPATCH_TRACE=1


 [call] op=[aten::zeros], key=[BackendSelect]
  [redispatch] op=[aten::zeros], key=[XLA]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[XLA]
   [call] op=[aten::zero_], key=[Functionalize]
    [call] op=[aten::zero_], key=[Meta]
    [call] op=[aten::zero], key=[XLA]
     [call] op=[aten::clone], key=[XLA]
     [call] op=[aten::zero_], key=[XLA]
    [call] op=[aten::_propagate_xla_data], key=[XLA]
 [call] op=[aten::zeros], key=[BackendSelect]
  [redispatch] op=[aten::zeros], key=[XLA]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[XLA]
   [call] op=[aten::zero_], key=[Functionalize]
    [call] op=[aten::zero_], key=[Meta]
    [call] op=[aten::zero], key=[XLA]
     [call] op=[aten::clone], key=[XLA]
     [call] op=[aten::zero_], key=[XLA]
    [call] op=[aten::_propagate_xla_data], key=[XLA]


In [3]:
%env TORCH_SHOW_DISPATCH_TRACE=1
@custom_op("xla::custom_max_pool", schema="(Tensor input) -> Tensor", mutates_args=())
def custom_max_pool(input: Tensor):
    return torch.max_pool2d(input)

env: TORCH_SHOW_DISPATCH_TRACE=1


In [4]:
%env TORCH_SHOW_DISPATCH_TRACE=1

test_lowering(lambda a, b: custom_linear_forward123(a, b))

env: TORCH_SHOW_DISPATCH_TRACE=1


 [callBoxed] op=[xla::custom_linear_forward123], key=[AutogradXLA]
  [redispatchBoxed] op=[xla::custom_linear_forward123], key=[Functionalize]
   [callBoxed] op=[xla::custom_linear_forward123], key=[XLA]
    [call] op=[aten::einsum], key=[XLA]


Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
OK


In [5]:
test_lowering(lambda a, b: torch.einsum('...n,mn->...m', a, b))

 [call] op=[aten::einsum], key=[AutogradXLA]


Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
OK
